# Data Science | Final Project | Group 02

## Identify epigenetic alterations associated with Alzheimer’s disease and classification of gene expressions between healthy and sick patients

#### Carmen Calle Huerta, Christina Kirschbaum, Pushpa Koirala, Melika Moradi

## Our Project

Alzheimer’s disease is the most prevalent kind of dementia and a fatal brain ailment. More study into this illness might lead to a better understanding of the condition and more effective treatment options. 

In this project, ChIP-seq data for H3K27ac, H3K9ac, H3K122ac and H3K4me1 as well as RNA-seq data from the from the lateral temporal lobe of the human brain for young healthy patients, old heathy patients and patients with Alzheimers disease will be analyzed and the differences between normal aging and cognitive impairment will be explored. The epigenomic or transcriptomic profiles will be analyzed to find relevant epigenetic alterations associated with the disease and help to better understand the molecular pathophysiology underlying.

Afterwards, with Machine Learning models the presence and absence of Alzheimer’s disease based on the data. The models will be built with Support Vector Machines and Random Forest.

Finally, the findings will be compared to them in related papers, we will look into relevant *in vivo* experiments with model organisms and use the Genome Browser to generate ChIP-seq tracks.

For our project we were inspirated by the paper:
Nativio R, Lan Y, Donahue G et al. ["An integrated multi-omics approach identifies
epigenetic alterations associated with Alzheimer’s disease."](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8098004/)

The data is derived from GEO, [Series GSE153875](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE153875).

## Import packages

First, we import some of the most frequently used packages in Python. [NumPy](https://numpy.org/doc/stable/) for working with arrays, matrices and linear algebra, [pandas](https://pandas.pydata.org/docs/) for data analysis and manipulation, [matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) for visualizations.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Integration and Preprocessing RNA-seq

In the next step, we import our RNA-seq data from the SubSeries GSE159699 of the SuperSeries GSE153875 with SRA-Toolkit and fastq-dump. To integrate this process into Python, the package subrocess is used.

In [2]:
import subprocess

# sra numbers from accession list of our RNA-seq data from SuperSeries GSE153875 / SubSeries GSE159699
sra_numbers = [
    "SRR12850830", "SRR12850831", "SRR12850832", "SRR12850833", "SRR12850834",
    "SRR12850835", "SRR12850836", "SRR12850837", "SRR12850838", "SRR12850839",
    "SRR12850840", "SRR12850841", "SRR12850842", "SRR12850843", "SRR12850844",
    "SRR12850845", "SRR12850846", "SRR12850847", "SRR12850848", "SRR12850849",
    "SRR12850850", "SRR12850851", "SRR12850852", "SRR12850853", "SRR12850854",
    "SRR12850855", "SRR12850856", "SRR12850857", "SRR12850858", "SRR12850859"
    ]

In [ ]:
# code from https://erilu.github.io/python-fastq-downloader/

# this will download the .sra files to ~/ncbi/public/sra/ 
for sra_id in sra_numbers:
    print ("Currently downloading: " + sra_id)
    prefetch = "prefetch " + sra_id
    print ("The command used was: " + prefetch)
    subprocess.call(prefetch, shell=True)

# this will extract the .sra files from above into a folder named 'fastq'
for sra_id in sra_numbers:
    print ("Generating fastq for: " + sra_id)
    fastq_dump = "fastq-dump --outdir fastq --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip ~/ncbi/public/sra/" + sra_id + ".sra"
    print ("The command used was: " + fastq_dump)
    subprocess.call(fastq_dump, shell=True)

After we downloaded the data, we will now preprocess it with STAR, an aligner for RNA-seq data mapping. Like in the paper of *Nativio et al.*, we will align our RNA-seq reads to the human reference genome (assembly GRCh37.75/hg19) using STAR with default parameters.
The [fasta](http://ftp.ensembl.org/pub/release-75/fasta/homo_sapiens/dna/) and the [gtf](http://ftp.ensembl.org/pub/release-75/gtf/homo_sapiens/) file used for the genomeDir argument was downloaded from Ensembl. 

This will create .sam files out of the .fastq files.

**Currently running**

In [4]:
path = "/mnt/c/Users/chris/OneDrive/Documents/GitHub/DataScience_finalProjekt_Group2"
path_genomeDir = "/mnt/c/Users/chris/OneDrive/Documents/GitHub/DataScience_finalProjekt_Group2/GenomeDir/"
path_fastq = "/mnt/c/Users/chris/OneDrive/Documents/GitHub/DataScience_finalProjekt_Group2/fastq/"

In [ ]:
# get Genome Index
genomeIndex = "STAR --runThreadN 2 --runMode genomeGenerate --genomeDir " + path_genomeDir + " --genomeFastaFiles " + path + "Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa --sjdbGTFfile " + path + "Homo_sapiens.GRCh37.75.gtf"
subprocess.call(genomeIndex, shell=True)

	STAR --runThreadN 2 --runMode genomeGenerate --genomeDir /mnt/c/Users/chris/GenomeDir/ --genomeFastaFiles /mnt/c/Users/chris/Downloads/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa --sjdbGTFfile /mnt/c/Users/chris/Downloads/Homo_sapiens.GRCh37.75.gtf
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Jun 14 22:46:07 ..... started STAR run
Jun 14 22:46:07 ... starting to generate Genome files
Jun 14 22:47:52 ..... processing annotations GTF
Jun 14 22:49:33 ... starting to sort Suffix Array. This may take a long time...
Jun 14 22:52:02 ... sorting Suffix Array chunks and saving them to disk...


In [ ]:
# Run mapping
for fastq in sra_numbers:
    print ("Currently aligning: " + fastq)
    gunzip = "gunzip " + path_fastq + fastq + "_pass.fastq.gz"
    subprocess.call(gunzip, shell=True)
    mapping = "STAR --runThreadN 2 --genomeDir " + path_genomeDir + " --readFilesIn " + fastq + "_pass.fastq"
    subprocess.call(mapping, shell=True)
    print("Done with " + fastq)
    break

## Data Integration for ChIP-seq

The .bed and .bw files for the ChIP-seq data were downloaded. They are available as supplementary files of the SuperSeries GSE153875 from GEO. We split them into folders for H3K27ac, H3K9ac, H3K122ac and H3K4me1 (and peaks) to get smaller sets.

**Where to store the data? ChIP-seq is overall 27GB, RNA-seq in FASTQ-format 50GB**

In [ ]:
# The following code can get the metadata (if needed) for the ChIP-seq data with the library GEOparse (similar to GEOquery).
import GEOparse

gse = GEOparse.get_GEO(geo="GSE153875", destdir="./")

# prints the metadata for the first sample
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    break

## Quality Control

## Exploratory Data Analysis